# Introduction

what we want to achieve

Importing libraries

In [21]:
import requests
from bs4 import BeautifulSoup
import re

In [22]:
def get_html(url, path):
    response = requests.get(url)
    with open(path, "w", encoding = "utf-8") as f:
        f.write(response.text)

In [23]:
url = "https://en.wikipedia.org/wiki/List_of_old-growth_forests"

In [24]:
get_html(url, path="HTML/wiki_old_forrest.html")

In [25]:
with open("HTML/wiki_old_forrest.html", "r", encoding = "utf-8") as f:
    html = f.read()

In [11]:
soup = BeautifulSoup(html, "html.parser")
tables = soup.find_all("table", attrs={"class": "wikitable sortable"})

In [26]:
data = {}
for table in tables:
    headings = table.find_previous(["h2", "h3"]).text
    print(headings)
    data[headings] = table

Africa
Asia
Australia
Europe
Canada
United States
Central America
Caribbean
South America


In [27]:
table = data["Australia"]
first_row = table.tr
for td in first_row:
    print(td.text)



Country



Area



Old-growth extent



WWF ecoregion



Old-growth forest type



In [28]:
columns = []
for td in first_row:
    if td.text.strip() != "":
        columns.append(td.text.strip())

columns

['Country',
 'Area',
 'Old-growth extent',
 'WWF ecoregion',
 'Old-growth forest type']

In [29]:
rows = table.tbody.find_all("tr")

In [30]:
def extract_row_data(columns, row):
    row_data = {}
    table_cells = row.find_all("td")
    for i in range(len(table_cells)):
        row_data[columns[i]] = table_cells[i]
    
    return row_data

In [31]:
def clean_row_data(row: dict):

    for k in row.keys():
        val = row[k]
        if re.match("\s", val.text):
            row[k] = "No data"

        links = val.find_all("a")

        for l in links:
            if l.get("title") is not None and "(page does not exist)" in l.get("title"):
                l.replace_with(l.text)

            if "cite" in l.get("href"):
                l.parent.decompose()

        if k == "Old-growth extent" and row[k] != "No data":
            data = row[k].text.strip()

            data = data.replace("\xa0", " ")
            
            # 2,000, 7,800,000
            data = re.search("\d+(?:,\d{3})*(?:\.\d*)? (?:hectares|square kilometres|ha|acres)", data).group()

            parent = row[k].parent
            row[k].decompose()

            new_tag = soup.new_tag("td")
            new_tag.string = data
            parent.append(new_tag)

            row[k] = new_tag

            
    return row

In [32]:
def prepare_table_data(columns, table):
    table_data = []

    rows = table.find_all("tr")
    rows.pop(0)

    for r in rows:
        r = extract_row_data(columns=columns, row=r)
        r = clean_row_data(r)
        table_data.append(r)

    return table_data

In [33]:
def prepare_all_tables(columns, data):
    for k in data.keys():
        data[k] = prepare_table_data(columns, data[k])

    return data

In [34]:
data = prepare_all_tables(columns, data)

AttributeError: 'NoneType' object has no attribute 'group'

# Data Analytics

In [ ]:
data["Australia"][3]["Old-growth extent"].text

In [ ]:
# How many of the listed forests are in France?
europe = data["Europe"]
france = [r for r in europe if "France" in r["Country"].text]
len(france)

In [ ]:
# How many of the listed forests are in Tasmania?
len([r for r in data["Australia"] if "Tasmania" in r["Area"].text])

In [ ]:
# In tasmania, of those that have data, what is the total area of these?
australia = data["Australia"]
tasmania = [r for r in data["Australia"] if "Tasmania" in r["Area"].text]
tasmania_area_data = [r for r in tasmania if r["Old-growth extent"] != "No data"]

total = 0
for r in tasmania_area_data:
    area = r["Old-growth extent"].text

    area = area.replace(",", "")
    val = re.search("\d*", area).group()
    val = float(val)

    if "square kilometres" in area:
        val = val * 100

    total += val

print("Total area for Tasmania forests: ", total, " ha")

## From the data of bulgaria's forests, what is the proportion of bulgarias total area that is covered by these?

In [35]:
bulgaria_rows = []
for row in data["Europe"]:
    if row["Country"].text.strip() == "Bulgaria":
        bulgaria_rows.append(row)

TypeError: string indices must be integers, not 'str'

In [ ]:
# get the link of the bulgaria article
bulgaria_link = "https://wikipedia.org" + bulgaria_rows[0]["Country"].a["href"]

In [ ]:
get_html(bulgaria_link, path="HTML/bulgaria.html")

In [ ]:
with open("HTML/bulgaria.html", "r", encoding = "utf-8") as f:
    html_bulgaria = f.read()

In [ ]:
bulgaria_soup = BeautifulSoup(html_bulgaria, "html.parser")
bulgaria_soup.title

In [ ]:
def get_bulgaria_area(tag):
    return tag.name == "td" and 'km' in tag.text and 'Total' in tag.parent.text

In [ ]:
km_tags = [t.text for t in bulgaria_soup.find_all(get_bulgaria_area)]

In [ ]:
area_tag = km_tags[0]
area_tag

In [ ]:
b_area = re.search("\d+(?:,\d{3})*(?:\.\d*)?", area_tag).group()
b_area = float(b_area.replace(',', ''))

In [ ]:
b_area = b_area * 100

In [ ]:
forest_total = 0
for row in bulgaria_rows:
    forest_data = row['Old-growth extent'].text
    forest_data = re.search("\d+(?:,\d{3})*(?:\.\d*)?", forest_data).group()
    forest_data = float(forest_data.replace(',', ''))

    forest_total += forest_data

forest_total

In [ ]:
print(f'Percentage of bulgarian land area accounted for old growth: {round((forest_total / b_area)*100, 5)}%')

### How many US states have forests with some variety of oak tree?

In [ ]:
us_table = data['United States']
states = set()

In [ ]:
us_oak = [r for r in us_table if "oak" in r["Old-growth forest type"].text]
len(us_oak)

In [ ]:
for r in us_table:
    f_type = r["Old-growth forest type"]
    if f_type is not None and f_type != 'No data':
        if 'oak' in f_type.text.lower():
            states.add(r["Country"].text.strip())
states